# PS2-1: Training Stability of Logistic Regression

## Introduction

The code in src/p01_lr.py trains a logistic regression algorithm on two different datasets, A and B in /data/ds1_{a/b}.csv. This problem investigates the difference in performance of the model on the two datasets.

## (A)
#### What is the main difference in training the model on dataset A vs B?

The training converges on dataset A while it seems to fail to converge on dataset B.

## (B)
#### Provide an explanation for the difference in behavior between the two datasets.

First, we make a plot of both datasets.


In [ ]:
from PS2.src import util
import matplotlib.pyplot as plt
import numpy as np

Xa , Ya = util.load_csv('.PS2/data/ds1_a.csv')
Xb , Yb = util.load_csv('.PS2/data/ds1_b.csv')



# Find the spread of the x_0 coordinate in both datasets.
x0a_spread = Xa[:,0].max() - Xa[:,0].min()
x1a_spread = Xa[:,1].max() - Xa[:,1].min()

x0b_spread = Xb[:,0].max() - Xb[:,0].min()
x1b_spread = Xb[:,1].max() - Xb[:,1].min()

# Calculate 20% of the spread as margin on either side 

# Calculate 20% of the spread as margin on either side 
x0a_margin = x0a_spread / 5
x1a_margin = x1a_spread / 5 


x0b_margin = x0b_spread / 5
x1b_margin = x1b_spread / 5 


fig , axa = plt.subplots()

# Set the x and y ranges
axa.set_xlim(Xa[:,0].min() - x0a_margin , Xa[:,0].max() + x0a_margin)
axa.set_ylim(Xa[:,1].min() - x1a_margin , Xa[:,1].max() + x1a_margin)

axa.set_title('plot of dataset a')

axa.plot(Xa[Ya == 1, 0], Xa[Ya == 1, 1], 'bx', linewidth=2, label='positive examples')
axa.plot(Xa[Ya == -1, 0], Xa[Ya == -1,1], 'go', linewidth=2, label='negative examples')

# Let's also plot the mean of the positive and negative examples.
m , n = Xa.shape
xa_pos_mean = 1/(2*m) * np.sum(Xa[Ya==1,:], axis=0)
xa_neg_mean = 1/(2*m) * np.sum(Xa[Ya==-1,:], axis=0)

axa.plot(xa_pos_mean[0], xa_pos_mean[1], 'yo', linewidth=10, label = 'positive mean')
axa.plot(xa_neg_mean[0], xa_neg_mean[1], 'yx', linewidth=10, label = 'negative mean')

axa.legend()


plt.savefig('ds1_a.png', format='png')
plt.show()


print(f'{100 * Xa[Ya==1,:].shape[0]/Xa.shape[0] } percent of dataset a examples are positive.')
print(f'The initial gradient of the loss function is {xa_pos_mean - xa_neg_mean} for dataset a')
